In [3]:
import pdfplumber
import pandas as pd
import numpy as np
import os

column_names = ['重量', '貨名', '單價', '單位', '倉位', '重量', '貨名', '單價', '單位', '倉位', '重量', '貨名', '單價', '單位', '倉位']

# 使用指定的列名初始化DataFrame
all_pages_df = pd.DataFrame(columns=column_names)
all_pages_df.insert(5, 'date',None)  # 在第5列之后插入 'date'
all_pages_df.insert(11, 'date2',None)  # 在第10列之后插入 'date'
all_pages_df.insert(17, 'date3', None)  # 在第15列之后插入 'date'

folder_path = "金山"

for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(folder_path, filename)

        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    # 动态获取列名
                    df = pd.DataFrame(table[1:], columns=column_names)
                    df.insert(5, 'date', filename)  # 在第5列之后插入 'date'
                    df.insert(11, 'date2', filename)  # 在第10列之后插入 'date'
                    df.insert(17, 'date3', filename)  # 在第15列之后插入 'date'
                    try:
                        all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                    except:
                        print(f"Error merging data from {filename},{page}")

print(len(all_pages_df))



4981


In [22]:
df1 = all_pages_df.iloc[:, :6]
df2 = all_pages_df.iloc[:, 6:12]
df3 = all_pages_df.iloc[:, 12:18]
df2.columns = df1.columns
df3.columns = df1.columns
df_concatenated_corrected = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)
df_concatenated_corrected=df_concatenated_corrected.dropna()
df = df_concatenated_corrected.applymap(lambda x: x.strip() if isinstance(x, str) else x).replace('', np.nan)
df_cleaned = df.dropna(subset=["貨名"], how='any').reset_index(drop=True)
df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\4189190492.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df_concatenated_corrected.applymap(lambda x: x.strip() if isinstance(x, str) else x).replace('', np.nan)
C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\4189190492.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [5]:
from datetime import datetime

def convert_date(date_str):
    date_formats = ['%d-%b-%y', '%d-%B-%y',"%d %B-%y"]
    for date_format in date_formats:
        try:
            date_obj = datetime.strptime(date_str, date_format)
            return date_obj.strftime('%Y-%m-%d')
        except ValueError:
            pass
    raise ValueError(f"Invalid date format for '{date_str}'")

df_cleaned['date'] = df_cleaned['date'].apply(lambda x: convert_date(x.replace('.pdf', '').replace(' updated', '')))

In [20]:
####brand
def get_brand_dict(brand_df):
    brand_df = brand_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    brand_df = brand_df.T
    new_columns = brand_df.iloc[0]
    df_new = brand_df[1:].reset_index(drop=True)
    df_new.columns = new_columns
    result_dict = df_new.to_dict('list')
    brand_dict = {k: [i for i in v if type(i) != float] for k, v in result_dict.items()}
    brand_dict['ibp'].append("i. b. p.")
    brand_dict['ibp'].append("i. b. p")
    brand_dict['frimesa'].append("frimsa")
    brand_dict['fortefrigo'] = None
    brand_dict['lkj'] = None
    brand_dict['nossobeef'] = None
    brand_dict['national'] = None
    brand_dict['swift'] = None
    brand_dict['kiwi'] = None
    return brand_dict

def get_brand_list(brand_dict):
    brand_list = []
    for key, values in brand_dict.items():
        brand_list.append(key)
        try:
            brand_list.extend(values)
        except:
            print(key)
    return brand_list

##origin
def get_origin_list():
    return ['西', '巴', '德', '丹', '波', '澳', '中', '匈牙利', '德', '俄', '愛', '墨', '荷', '阿', '荷 蘭', '挪 威', '英 國', '英', '紐', '法 國', '爱', '加', '美', '德國']

##product
def get_product_dict(product_df):
    df_clean = product_df.dropna(axis=1, how='all')
    df_melted = df_clean.melt(id_vars=['category'], value_name='product').dropna(subset=['product']).drop('variable', axis=1)
    df_grouped = df_melted.groupby('category')['product'].apply(list).reset_index()
    product_dict = df_grouped.set_index('category')['product'].to_dict()
    product_dict['Pork'].pop()
    product_dict['Pork'].remove('筒骨')
    product_list = []
    for key, values in product_dict.items():
        for value in values:
            product_list.append(value)
    value = product_dict.pop('Meatball')
    product_dict['Meatball'] = value
    product_dict['Beef'].append('牛')
    product_dict['Chicken'].append('雞')
    product_dict['Chicken'].append('鸡')
    product_dict['Lamb'].append('羊')
    product_dict['Pork'].append('豬')
    product_dict['Poultry'].append('鴨')
    return product_dict, product_list

##grade
def get_grade_dict(grade_df):
    grade_df = grade_df.T
    grade_df = grade_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    new_columns = grade_df.iloc[0]
    df_new = grade_df[1:].reset_index(drop=True)
    df_new.columns = new_columns
    result_dict = df_new.to_dict('list')
    grade_dict = {k: [i for i in v if type(i) != float] for k, v in result_dict.items()}
    del grade_dict['m']
    del grade_dict['s']
    del grade_dict['l']
    del grade_dict['a']
    grade_list = []
    for key, values in grade_dict.items():
        grade_list.append(key)
        grade_list.extend(values)
    return grade_dict, grade_list

## Country
def get_country_dict(country_df):
    country_df = country_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    country_df = country_df.T
    new_columns = country_df.iloc[0]
    df_new = country_df[1:].reset_index(drop=True)
    df_new.columns = new_columns
    result_dict = df_new.to_dict('list')
    country_dict = {k: [i for i in v if type(i) != float] for k, v in result_dict.items()}
    country_dict.pop('standard_country')
    country_list = []
    for key, values in country_dict.items():
        country_list.append(key)
        try:
            country_list.extend(values)
        except:
            print(key)
    return country_dict, country_list

## warehouse
def get_warehouse_dict(warehouse_df):
    warehouse_df = warehouse_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    warehouse_df = warehouse_df.T
    new_columns = warehouse_df.iloc[0]
    df_new = warehouse_df[1:].reset_index(drop=True)
    df_new.columns = new_columns
    result_dict = df_new.to_dict('list')
    warehouse_dict = {k: [i for i in v if type(i) != float] for k, v in result_dict.items()}
    warehouse_list = []
    for key, values in warehouse_dict.items():
        warehouse_list.append(key)
        try:
            warehouse_list.extend(values)
        except:
            print(key)
    return warehouse_dict, warehouse_list

In [7]:
brand_df = pd.read_csv("brand_conversion_table.csv")
brand_dict = get_brand_dict(brand_df)
brand_list = get_brand_list(brand_dict)

origin_list = get_origin_list()

product_df = pd.read_csv("Product_conversion_table.csv")
product_dict, product_list = get_product_dict(product_df)

grade_df = pd.read_csv("grade_conversion.csv")
grade_dict, grade_list = get_grade_dict(grade_df)

country_df = pd.read_csv("country_conversion.csv")
country_dict, country_list = get_country_dict(country_df)

warehouse_df = pd.read_csv("warehouse_conversion.csv")
warehouse_dict, warehouse_list = get_warehouse_dict(warehouse_df)

fortefrigo
lkj
nossobeef
national
swift
kiwi


C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\1413608902.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  brand_df = brand_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\1413608902.py:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\1413608902.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  grade_df = grade_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\1413608902.py:64: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_19020\1413608902.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  country_df = country

In [24]:
def get_origin(row, origin_list):
    for origin_item in origin_list:
        if origin_item in row['貨名']:
            return origin_item
    return None

def get_brand(row, brand_list):
    for brand_item in brand_list:
        if brand_item in row['貨名']:
            parts = row['貨名'].split(brand_item, 1)
            return brand_item
    return None

def get_warehouse(row, warehouse_list):
    for warehouse_item in warehouse_list:
        if warehouse_item in row['倉位']:
            parts = row['倉位'].split(warehouse_item, 1)
            return warehouse_item
    return None

def get_name(row, brand, product_list):
    for product_item in product_list:
        if product_item in row['貨名']:
            return product_item
    if brand:
        parts = row['貨名'].split(brand, 1)
        name = parts[-1].strip() if len(parts) > 1 else row['貨名'].strip()
    else:
        name = row['貨名'].strip()
    return name

def get_grade(row, grade_list):
    grade = ""
    for grade_item in grade_list:
        if grade_item in row['貨名']:
            grade += grade_item + ','
    return grade.rstrip(',')

def get_specs(row):
    specs = row['grade'].split(',')
    spec1 = specs[0] if len(specs) > 0 else ''
    spec2 = specs[1] if len(specs) > 1 else ''
    spec3 = specs[2] if len(specs) > 2 else ''
    spec4 = specs[3] if len(specs) > 3 else ''
    spec5 = specs[4] if len(specs) > 4 else ''
    return pd.Series([spec1, spec2, spec3, spec4, spec5])

def get_category(row, product_dict):
    for category, products in product_dict.items():
        for product in products:
            if product in row['name']:
                return category
    return ''

def get_product_name(row):
    return row['貨名'].replace(row['Origin'], '')

def get_supplier(row):
    return '金山'

In [25]:
df_cleaned['Origin'] = df_cleaned.apply(lambda row: get_origin(row, origin_list), axis=1)
df_cleaned['brand'] = df_cleaned.apply(lambda row: get_brand(row, brand_list), axis=1)
df_cleaned['倉位'] = df_cleaned.apply(lambda row: get_warehouse(row, warehouse_list), axis=1)
df_cleaned['name'] = df_cleaned.apply(lambda row: get_name(row, row['brand'], product_list), axis=1)
df_cleaned['grade'] = df_cleaned.apply(lambda row: get_grade(row, grade_list), axis=1)
df_cleaned["grade"] = df_cleaned["grade"].str.replace("choice,ch", "choice")
df_cleaned[['spec1', 'spec2', 'spec3', 'spec4', 'spec5']] = df_cleaned.apply(get_specs, axis=1, result_type='expand')
df_cleaned['category'] = df_cleaned.apply(lambda row: get_category(row, product_dict), axis=1)
df_cleaned['product_name'] = df_cleaned.apply(get_product_name, axis=1)
df_cleaned['supplier'] = df_cleaned.apply(get_supplier, axis=1)


In [26]:
reverse_brand_dict = {v: k for k, values in brand_dict.items() if values is not None for v in values}
df_cleaned['brand'] = df_cleaned['brand'].map(reverse_brand_dict).fillna(df_cleaned['brand'])

reverse_grade_dict = {v: k for k, values in grade_dict.items() if values is not None for v in values}
df_cleaned['spec1'] = df_cleaned['spec1'].map(reverse_grade_dict).fillna(df_cleaned['spec1'])
df_cleaned['spec2'] = df_cleaned['spec2'].map(reverse_grade_dict).fillna(df_cleaned['spec2'])
df_cleaned['spec3'] = df_cleaned['spec3'].map(reverse_grade_dict).fillna(df_cleaned['spec3'])
df_cleaned['spec4'] = df_cleaned['spec4'].map(reverse_grade_dict).fillna(df_cleaned['spec4'])
df_cleaned['spec5'] = df_cleaned['spec5'].map(reverse_grade_dict).fillna(df_cleaned['spec5'])

reverse_country_dict = {v: k for k, values in country_dict.items() if values is not None for v in values}
df_cleaned['Origin'] = df_cleaned['Origin'].map(reverse_country_dict).fillna(df_cleaned['Origin'])

reverse_warehouse_dict = {v: k for k, values in warehouse_dict.items() if values is not None for v in values}
df_cleaned['貨名'] = df_cleaned['貨名'].map(reverse_warehouse_dict).fillna(df_cleaned['貨名'])


In [27]:
df_cleaned = df_cleaned.rename(columns={
    'product_name': 'productName',
    '重量': 'packing',
    'Origin': 'origin',
    'brand': 'brand',
    'date': 'effectiveDate',
    'spec1': 'spec1',
    'spec2': 'spec2',
    'spec3': 'spec3',
    'spec4': 'spec4',
    'spec5': 'spec5',
    '單價': 'price',
    '單位': 'weightUnit',
    '倉位': 'warehouse',
    'name': 'productTag',
    'supplier': 'supplier'
})

new_column_order = [
    'productName', 'supplier', 'category', 'packing', 'origin',
    'brand', 'effectiveDate', 'spec1', 'spec2', 'spec3', 'spec4',
    'spec5', 'price', 'weightUnit', 'warehouse', 'productTag'
]

df_cleaned = df_cleaned[new_column_order]

In [28]:
df_cleaned

,productName,supplier,category,packing,origin,brand,effectiveDate,spec1,spec2,spec3,spec4,spec5,price,weightUnit,warehouse,productTag
0,dom glutao 牛前筒骨,金山,Beef,抄,巴西,dom glutao,10-jul-23.pdf,,,,,,$ 5 .2,磅,其士,牛前筒骨
1,牛坑腩 uk-9012,金山,Beef,抄,英國,hewitt,10-jul-23.pdf,,,,,,$ 8 .0,磅,其士,牛坑腩
2,mataboi 牛坑腩 80vl p-3112,金山,Beef,抄,巴西,boi brasil,10-jul-23.pdf,80vl,,,,,$ 1 3.5,磅,其士,牛坑腩
3,plena 牛坑腩 80vl,金山,Beef,抄,巴西,plena,10-jul-23.pdf,80vl,,,,,$ 1 3.5,磅,其士,牛坑腩
4,excel 牛下肩肉眼卷(1x1) prime p-93,金山,Beef,抄,加拿大,excel,10-jul-23.pdf,prime,1x1,pr,,,$ 4 0.0,磅,其士,牛下肩肉眼卷
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9052,aurora 40+ 雞腳,金山,Chicken,12kg,巴西,aurora,9-oct-23.pdf,40+,,,,,$ 4 .2,磅,其士,雞腳
9053,jagua 50+ 雞腳,金山,Chicken,15kg,巴西,jagua,9-oct-23.pdf,50+,,,,,$ 4 .8,磅,其士,雞腳
9054,seara 火雞腎,金山,Chicken,10kg,巴西,seara,9-oct-23.pdf,,,,,,$ 1 3.5,磅,其士,雞腎
9055,鴨舌 hu332,金山,Poultry,10kg,匈牙利,None,9-oct-23.pdf,u3,,,,,$ 5 2.0,磅,其士,鴨舌
